In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# графики
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

# для лемматизации
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

import warnings
warnings.filterwarnings('ignore',category=DeprecationWarning)

from nltk.corpus import stopwords
stop_words = stopwords.words('russian')
stop_words.extend(['такой', 'который', 'какой', 'свой', 'мой', 'наш', 'этот', 'очень'])
stop_words.extend(['тысяча', 'сотня', 'всякий', 'любой'])

# импорт датасета
#df = pd.read_csv('C:\\Users\\vinov\\OneDrive\\Документы\\Универ\\Диплом\\Код\\archive\\lenta-ru-news.csv', low_memory = False, dtype = str)
df = pd.read_csv('C:\\Users\\vinov\\OneDrive\\Документы\\Универ\\Диплом\\Код\\archive\\labeled.csv', low_memory = False, dtype = str)
#print(df.topic.unique())
df.head()

# конвертировать в список
data = df.comment.values.tolist()
#data = df.text.values.tolist()

# удалить эл. адреса
data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]

# удалить символы новой строки
data = [re.sub(r'\s+', ' ', sent) for sent in data]

# удалить кавычки
data = [re.sub(r'\"', '', sent) for sent in data]

pprint(data[:2])

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
print(data_words[:2])

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100) 
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)


# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatize(texts, allowed_postags = ['NOUN', 'ADJF', 'INFN', 'PRTF', 'GRND', 'ADV']):
    texts_out = []
    for sent in texts:
        butyavka = morph.parse(str(sent))[0]
        if butyavka.tag.POS in allowed_postags:
            texts_out.append(butyavka.lexeme)
    return texts_out

def lemmatize(texts, allowed_postags = ['NOUN', 'ADJF', 'INFN', 'PRTF', 'GRND', 'ADVB']):
    texts_out = []
    for sent in texts:
        to1 = []
        for part in sent:
            butyavka = morph.parse(part)[0]
            if butyavka.tag.POS in allowed_postags:
                to1.append(butyavka.normal_form)
        texts_out.append(to1)
    return texts_out



# биграмы
data_words_bigrams = make_bigrams(data_words)

data_lemmatized = lemmatize(data_words_bigrams)
print(data_lemmatized[:2])

# убрать стоп-слова
data_nonstop = remove_stopwords(data_lemmatized)

# Create Dictionary
id2word = corpora.Dictionary(data_nonstop)
# Create Corpus
texts = data_nonstop
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:2])

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_nonstop, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

['Верблюдов-то за что? Дебилы, бл... ',
 'Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. '
 'Если бы хохлов не было, кисель их бы придумал. ']
[['верблюдов', 'то', 'за', 'что', 'дебилы', 'бл'], ['хохлы', 'это', 'отдушина', 'затюканого', 'россиянина', 'мол', 'вон', 'хохлов', 'еще', 'хуже', 'если', 'бы', 'хохлов', 'не', 'было', 'кисель', 'их', 'бы', 'придумал']]
[['верблюдов', 'дебил'], ['хохол', 'отдушина', 'затюканый', 'россиянин', 'вон', 'хохлов', 'хохлов', 'кисель']]
[[(0, 1), (1, 1)], [(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 1)]]

Perplexity:  -22.77133982285081

Coherence Score:  0.43471454266956516


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.449699  0.099359       1        1  26.973626
11    -0.055350 -0.120912       2        1   9.915853
14    -0.066893 -0.333258       3        1   9.569818
0     -0.018963 -0.002258       4        1   7.857543
19    -0.019830 -0.004087       5        1   6.142588
10     0.004921  0.016658       6        1   5.668447
15     0.003747  0.016006       7        1   5.368575
16     0.013149  0.020337       8        1   4.748764
6      0.026597  0.024572       9        1   3.892834
12     0.022522  0.023609      10        1   3.790972
9      0.044224  0.027027      11        1   2.785254
5      0.044460  0.027003      12        1   2.376838
18     0.051957  0.026764      13        1   2.102823
13     0.051536  0.026848      14        1   1.730533
4      0.055989  0.026140      15        1   1.712342
17     0.056476  0.026090      16        1   1.632883
3      0.057950  0.025645      17        1   1.557054
1      0.058159  0.025502      18        1   0.945386
2      0.055837  0.026127      19        1   0.875779
7      0.063210  0.022827      20        1   0.352086, topic_info=          Term         Freq        Total Category  logprob  loglift
70        весь  8831.000000  8831.000000  Default   30.000  30.0000
113        год  3180.000000  3180.000000  Default   29.000  29.0000
157     вообще  1736.000000  1736.000000  Default   28.000  28.0000
141      самый  1841.000000  1841.000000  Default   27.000  27.0000
1010     время  1697.000000  1697.000000  Default   26.000  26.0000
...        ...          ...          ...      ...      ...      ...
1543     запад     0.016568     1.656972  Topic20  -10.494   1.0438
2162  работать     0.016568     1.656679  Topic20  -10.494   1.0439
1587     видос     0.016568     1.656565  Topic20  -10.494   1.0440
2271     тариф     0.016567     1.656385  Topic20  -10.494   1.0441
4131    машина     0.016568     1.656840  Topic20  -10.494   1.0438

[753 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
35       11  0.969480  абсолютный
84        4  0.988831       автор
919      12  0.970169    аргумент
1357     12  0.968990       армия
194      20  0.836848      бандит
...     ...       ...         ...
1464      8  0.986850         щас
488       9  0.961389       южный
866      16  0.953314        явно
504       6  0.989486        язык
282       8  0.970647        ясно

[401 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 12, 15, 1, 20, 11, 16, 17, 7, 13, 10, 6, 19, 14, 5, 18, 4, 2, 3, 8])